<a href="https://colab.research.google.com/github/GoogTech/langchain-tutorials/blob/master/Chatbots/Build_a_Chatbot.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [36]:
!pip install langchain langchain-core langgraph langchain-openai

In [4]:
!pip show langchain langchain_openai

Name: langchain
Version: 0.3.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-openai
Version: 0.3.4
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


# Quickstart

In [16]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from google.colab import userdata
import os

model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

model.invoke([HumanMessage(content="Hi! I'm Hack")])

AIMessage(content='Hi, Hack! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-47a18687-01a7-42db-992b-2d7942b04c80-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal data about users unless it's shared with me in the course of our conversation. If you'd like me to know your name, feel free to tell me!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 11, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1b129be-bc2f-478e-89cb-1d2cdcc716eb-0', usage_metadata={'input_tokens': 11, 'output_tokens': 40, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 33, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f8abb00a-a5f0-4798-806a-4c0a1eb0e853-0', usage_metadata={'input_tokens': 33, 'output_tokens': 12, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Message persistence

In [27]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [30]:
config = {"configurable": {"thread_id": "thread_id_123"}}

query = "Hi! I'm Hack"

input_messages = HumanMessage(content=query)
response = app.invoke({"messages": input_messages}, config)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi, Hack! It's great to meet you. How can I assist you today?


In [31]:
config = {"configurable": {"thread_id": "thread_id_123"}}

query = "What's my name?"

input_messages = HumanMessage(content=query)
response = app.invoke({"messages": input_messages}, config)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Hack! How can I help you today, Hack?


In [35]:
config = {"configurable": {"thread_id": "thread_id_789"}} # different thread_id

query = "What's my name?"

input_messages = HumanMessage(content=query)
response = app.invoke({"messages": input_messages}, config)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have any information about your name. If you'd like to share it, I can certainly address you by it!


# Prompt templates

In [41]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
  # response = model.invoke(state["messages"])
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [42]:
config = {"configurable": {"thread_id": "thread_id_666"}}

query = "Hi! I'm Hack"

input_messages = HumanMessage(content=query)
response = app.invoke({"messages": input_messages}, config)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Hack! What be bringin' ye to these digital seas today? Be ye lookin' for treasure, knowledge, or a hearty yarn?


In [43]:
config = {"configurable": {"thread_id": "thread_id_666"}}

query = "hat is my name?"

input_messages = HumanMessage(content=query)
response = app.invoke({"messages": input_messages}, config)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Hack, savvy? A fine name fer a landlubber or a scallywag alike! What else can I help ye with, matey?


In [44]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# Define the prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

# Define a new graph
workflow = StateGraph(state_schema=State)

# Define the function that calls the model
def call_model(state: State):
  # response = model.invoke(state["messages"])
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [46]:
config = {"configurable": {"thread_id": "abc888"}}

query = "Hi! I'm Hack"
language = "Chinese"

input_messages = [HumanMessage(query)]
response = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，Hack！有什么我可以帮助你的吗？


In [47]:
config = {"configurable": {"thread_id": "abc888"}}

query = "What is my name?"
language = "Chinese"

input_messages = [HumanMessage(query)]
response = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是Hack。有什么其他问题吗？
